In [1]:
import skmob
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import geocoder
import datetime

tqdm.pandas()

c:\users\askib\pycharmprojects\mobility\venv\lib\site-packages\tqdm\std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
path = "../data/raw/KBR_Gadow_2019_bez_hasla.xlsx"

In [11]:
df = pd.read_excel(path, header=[0,1])

In [12]:
columns = list(df.columns)[199:]
columns.append(list(df.columns)[0])
# df = df[l]
df = df[columns]

In [13]:
df.columns = df.columns.droplevel(1)
df.rename(columns={df.columns[270]:'ID odpowiedzi'}, inplace=True)
df.columns

Index(['ID podróży (1)', 'Data rozpoczecia podróży (1)',
       'Czas trwania podróży [s] (1)', 'Środek transportu (1)',
       'Typ miejsca startu (1)', 'Typ miejsca zakończenia (1)',
       'Miejsce startu (1)', 'Numer regionu miejsca startu (1)',
       'Numer regionu miejsca zakończenia (1)', 'Miejsce zakończenia (1)',
       ...
       'Data rozpoczecia podróży (27)', 'Czas trwania podróży [s] (27)',
       'Środek transportu (27)', 'Typ miejsca startu (27)',
       'Typ miejsca zakończenia (27)', 'Miejsce startu (27)',
       'Numer regionu miejsca startu (27)',
       'Numer regionu miejsca zakończenia (27)', 'Miejsce zakończenia (27)',
       'ID odpowiedzi'],
      dtype='object', length=271)

In [14]:
new_columns = [
    'user_id',
    'travel_id',
    'start_date',
    'duration_time',
    'mean_of_transport',
    'start_place_type',
    'finish_place_type',
    'start_place',
    'start_region',
    'residence_region',
    'finish_place'
]

travels_df = pd.DataFrame(columns=new_columns)

for i in tqdm(range(1, 28)):
    columns = [
        'ID odpowiedzi',
        f'ID podróży ({i})',
        f'Data rozpoczecia podróży ({i})',
        f'Czas trwania podróży [s] ({i})',
        f'Środek transportu ({i})',
        f'Typ miejsca startu ({i})',
        f'Typ miejsca zakończenia ({i})',
        f'Miejsce startu ({i})',
        f'Numer regionu miejsca startu ({i})',
        f'Numer regionu miejsca zakończenia ({i})',
        f'Miejsce zakończenia ({i})'
    ]

    sub_df = df[columns]
    sub_df = sub_df.dropna()
    sub_df.columns = new_columns

    travels_df = travels_df.append(sub_df)

100%|██████████| 27/27 [00:00<00:00, 28.17it/s]


In [15]:
travels_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 0 to 259
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            1277 non-null   object 
 1   travel_id          1277 non-null   object 
 2   start_date         1277 non-null   object 
 3   duration_time      1277 non-null   object 
 4   mean_of_transport  1277 non-null   object 
 5   start_place_type   1277 non-null   object 
 6   finish_place_type  1277 non-null   object 
 7   start_place        1277 non-null   object 
 8   start_region       1277 non-null   float64
 9   residence_region   1277 non-null   float64
 10  finish_place       1277 non-null   object 
dtypes: float64(2), object(9)
memory usage: 119.7+ KB


In [29]:
tdf = pd.DataFrame(columns=['travel_id', 'lat', 'long', 'time', 'mean_of_transport'])

for index, row in tqdm(travels_df.iterrows(), total=travels_df.shape[0]):
    g_start = geocoder.osm(row['start_place'])
    g_finish = geocoder.osm(row["finish_place"])

    try:
        tdf = tdf.append({
            'travel_id': row['travel_id'],
            'lat': g_start.osm['x'],
            'long': g_start.osm['y'],
            'time': datetime.datetime.strptime(row["start_date"], '%Y-%m-%d %H:%M:%S UTC'),
            'mean_of_transport': row['mean_of_transport']
        }, ignore_index=True)

        tdf = tdf.append({
            'travel_id': row['travel_id'],
            'lat': g_finish.osm['x'],
            'long': g_finish.osm['y'],
            'time': datetime.datetime.strptime(row["start_date"], '%Y-%m-%d %H:%M:%S UTC')
 + datetime.timedelta(0,int(row["duration_time"])),
            'mean_of_transport': row['mean_of_transport']
        }, ignore_index=True)
    except Exception as e:
        print()
        print(e)
        print(g_start.osm)
        print(g_finish.osm)

100%|██████████| 1277/1277 [25:01<00:00,  1.18s/it]



'NoneType' object is not subscriptable
{'x': 16.9626873, 'y': 51.1283708, 'addr:housenumber': '19', 'addr:street': 'Jerzego Bajana', 'addr:city': 'Wrocław', 'addr:state': 'województwo dolnośląskie', 'addr:country': 'Polska', 'addr:postal': '54-129'}
None

'NoneType' object is not subscriptable
{'x': 16.9639509, 'y': 51.1265349, 'addr:housenumber': '28', 'addr:street': 'Jerzego Bajana', 'addr:city': 'Wrocław', 'addr:state': 'województwo dolnośląskie', 'addr:country': 'Polska', 'addr:postal': '54-129'}
None

'NoneType' object is not subscriptable
None
{'x': 17.0811457, 'y': 51.1336137, 'addr:housenumber': '18-20', 'addr:street': 'Szpacza', 'addr:city': 'Wrocław', 'addr:state': 'województwo dolnośląskie', 'addr:country': 'Polska', 'addr:postal': '51-417'}

'NoneType' object is not subscriptable
None
{'x': 17.01338, 'y': 51.1110657, 'addr:street': 'Braniborska', 'addr:city': 'Wrocław', 'addr:state': 'województwo dolnośląskie', 'addr:country': 'Polska', 'addr:postal': '53-680'}

'NoneType'

In [30]:

tdf.to_csv("../data/interim/flow_matrix_gadow.csv")

In [4]:
df = pd.read_csv("../data/interim/flow_matrix_gadow.csv")

In [17]:
travels_df['start_place_type'].value_counts()

home                    507
intermediate            248
work                    219
other                   148
shop                     94
school_or_university     48
park                      9
office                    4
Name: start_place_type, dtype: int64

In [19]:
travels_df['finish_place_type'].value_counts()

home                    316
work                    288
intermediate            251
other                   208
shop                    122
school_or_university     63
park                     21
office                    8
Name: finish_place_type, dtype: int64